---
# Setup

In [20]:
import os
import sys
import logging
import datetime
import calendar
from typing import (
    List,
    Dict,
    Tuple,
    Any,
    Optional,
    Callable,
    Iterable,
)

import dateutil
import pandas as pd
import numpy as np

## Logging

In [18]:
logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger("analysis")

## PYTHONPATH

In [11]:
sys.path.append(f"{os.getcwd()}/../../../lib")

In [27]:
%load_ext autoreload
%autoreload 2

from util_datetime import (
    get_datetime_components,
    parse_date_string,
    get_dates_from_string,
    has_date_in_string,
    parse_datetime_string,
    get_epoch_from_datetime,
    get_epoch_from_string,
    get_seconds_between_datetimes,
    get_datetime_after_duration,
    get_elapsed_time,
    get_holidays,
    get_cyclic_time_of_day,
    get_cyclic_day_of_week,
    get_cyclic_month_of_year,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
# Data

In [23]:
df = pd.read_json("../data/september.json")

In [24]:
df

,facility,timeStart,processTime,supplier,suppliedM3,recoveredM3,date,timeEnd,supplierCode
0,Bundaberg,9/1/22 8:16 AM,4:05,Mary,5.09,4.13,NaT,NaN,NaN
1,Newcastle,8:29:00 AM,NaN,NaN,2.00,1.55,2022-09-01,9:07:00 AM,har
2,Newcastle,9:27:00 AM,NaN,NaN,6.80,4.15,2022-09-01,11:28:00 AM,dic
3,Newcastle,11:38:00 AM,NaN,NaN,1.95,1.55,2022-09-01,12:21:00 PM,har
4,Bundaberg,9/1/22 12:34 PM,1:50,Mary Therese,3.78,2.56,NaT,NaN,NaN
5,Newcastle,12:40:00 PM,NaN,NaN,3.95,2.55,2022-09-01,2:04:00 PM,tom
6,Newcastle,2:25:00 PM,NaN,NaN,5.30,3.10,2022-09-01,4:29:00 PM,dic
7,Bundaberg,9/1/22 2:48 PM,1:20,Mary Therese,3.55,2.59,NaT,NaN,NaN
8,Newcastle,8:16:00 AM,NaN,NaN,4.20,3.65,2022-09-02,10:55:00 AM,tom
9,Bundaberg,9/2/22 8:27 AM,1:25,Mary,5.02,4.13,NaT,NaN,NaN


In [25]:
df[df.isna().any(axis=1)]

,facility,timeStart,processTime,supplier,suppliedM3,recoveredM3,date,timeEnd,supplierCode
0,Bundaberg,9/1/22 8:16 AM,4:05,Mary,5.09,4.13,NaT,NaN,NaN
1,Newcastle,8:29:00 AM,NaN,NaN,2.00,1.55,2022-09-01,9:07:00 AM,har
2,Newcastle,9:27:00 AM,NaN,NaN,6.80,4.15,2022-09-01,11:28:00 AM,dic
3,Newcastle,11:38:00 AM,NaN,NaN,1.95,1.55,2022-09-01,12:21:00 PM,har
4,Bundaberg,9/1/22 12:34 PM,1:50,Mary Therese,3.78,2.56,NaT,NaN,NaN
5,Newcastle,12:40:00 PM,NaN,NaN,3.95,2.55,2022-09-01,2:04:00 PM,tom
6,Newcastle,2:25:00 PM,NaN,NaN,5.30,3.10,2022-09-01,4:29:00 PM,dic
7,Bundaberg,9/1/22 2:48 PM,1:20,Mary Therese,3.55,2.59,NaT,NaN,NaN
8,Newcastle,8:16:00 AM,NaN,NaN,4.20,3.65,2022-09-02,10:55:00 AM,tom
9,Bundaberg,9/2/22 8:27 AM,1:25,Mary,5.02,4.13,NaT,NaN,NaN


In [26]:
def extract_start_time(row):
    logger.info("row type[%s] row[%s]", type(row), row)

    start_date_time: datetime.datetime

    if has_date_in_string(row['timeStart']):
        # --------------------------------------------------------------------------------
        # timeStart already includes date.
        # --------------------------------------------------------------------------------
        start_date_time = parse_datetime_string(row['timeStart'])
        logger.debug("extract_start_time(): start_date_time is [%s]", start_date_time)
    
        assert 
    
    if dates is not None and len(dates) > 0:
        start_date_time = parse_datetime_string(row['timeStart'])
    else:
        

IndentationError: expected an indented block (1210609010.py, line 11)

In [ ]:
df.apply()